# Domain Adaptation Via Activation Shaping

**Teacher assistant** Iurada Leonardo

**Students**

- Bar Giorgio
- Distefano Giuseppe
- Incaviglia Salvatore

## Code

### Libraries

In [1]:
!pip install torch torchvision tqdm torchmetrics

import torch

import torch.backends.mps
from argparse import ArgumentParser

import os
import torchvision.transforms as T

from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CIFAR10

import numpy as np
import random
from PIL import Image

import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights

import torch.nn.functional as F
from torchmetrics import Accuracy
from tqdm import tqdm

import logging
import warnings

import argparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 10.4 MB/s eta 0:00:00


### Setup PACS dataset and environment

In [2]:
# Download PACS Dataset Images and Labels
!git clone https://github.com/MachineLearning2020/Homework3-PACS/
!git clone https://github.com/silvia1993/DANN_Template/

# Setup data
!rm -rf data || true
!rm -rf record || true
!mkdir data
!mkdir data/kfold
!cp -r Homework3-PACS/PACS/ data/kfold
!cp DANN_Template/txt_lists/*.txt data
!rm -rf Homework3-PACS/
!rm -rf DANN_Template/
!rm -rf sample_data || true

Cloning into 'Homework3-PACS'...
remote: Enumerating objects: 10032, done.
remote: Total 10032 (delta 0), reused 0 (delta 0), pack-reused 10032
Receiving objects: 100% (10032/10032), 174.13 MiB | 34.88 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (9993/9993), done.
Cloning into 'DANN_Template'...
remote: Enumerating objects: 23, done.
remote: Total 23 (delta 0), reused 0 (delta 0), pack-reused 23
Receiving objects: 100% (23/23), 33.86 KiB | 8.47 MiB/s, done.
Resolving deltas: 100% (5/5), done.


### Globals

#### globals

In [3]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

CONFIG = dotdict({})

if torch.cuda.is_available():
    CONFIG.device = 'cuda'
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    CONFIG.device = 'mps'
else:
    CONFIG.device = 'cpu'

CONFIG.dtype = torch.float32

#### parse args

In [4]:
def _clear_args(parsed_args):
    parsed_args.experiment_args = eval(parsed_args.experiment_args)
    parsed_args.dataset_args = eval(parsed_args.dataset_args)
    return parsed_args


def parse_arguments():
    parser = ArgumentParser()

    parser.add_argument('--seed', type=int, default=0, help='Seed used for deterministic behavior')
    parser.add_argument('--test_only', action='store_true', help='Whether to skip training')
    parser.add_argument('--cpu', action='store_true', help='Whether to force the usage of CPU')

    parser.add_argument('--experiment', type=str, default='baseline')
    parser.add_argument('--experiment_name', type=str, default='baseline')
    parser.add_argument('--experiment_args', type=str, default='{}')
    parser.add_argument('--dataset_args', type=str, default='{}')

    parser.add_argument('--batch_size', type=int, default=128)
    parser.add_argument('--epochs', type=int, default=30)
    parser.add_argument('--num_workers', type=int, default=5)
    parser.add_argument('--grad_accum_steps', type=int, default=1)

    return _clear_args(parser.parse_args())

#### Global variables

In [5]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES = 7
BATCH_SIZE = 256
LR = 1e-3               # The initial Learning Rate
MOMENTUM = 0.9          # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5     # Regularization, you can keep this at the default
NUM_EPOCHS = 30         # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20          # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1             # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

### Dataset

#### utils

In [6]:
class BaseDataset(Dataset):
    def __init__(self, examples, transform):
        self.examples = examples
        self.T = transform


    def __len__(self):
        return len(self.examples)


    def __getitem__(self, index):
        x, y = self.examples[index]
        x = Image.open(x).convert('RGB')
        x = self.T(x).to(CONFIG.dtype)
        y = torch.tensor(y).long()
        return x, y

######################################################

class DomainAdaptationDataset(Dataset):
    def __init__(self, source_examples, target_examples, transform):
        self.source_examples = source_examples
        self.target_examples = target_examples
        self.T = transform


    def __len__(self):
        return len(self.source_examples)


    def __getitem__(self, index):
        src_x, src_y = self.source_examples[index]
        src_x = Image.open(src_x).convert('RGB')
        src_x = self.T(src_x).to(CONFIG.dtype)
        src_y = torch.tensor(src_y).long()

        # randomly sample 'target_examples' to obtain targ_x
        target_idx = torch.randint(len(self.target_examples), size=(1,)).item()
        targ_x, _ = self.target_examples[target_idx]
        targ_x = Image.open(targ_x).convert('RGB')
        targ_x = self.T(targ_x).to(CONFIG.dtype)

        return src_x, src_y, targ_x

######################################################

# [OPTIONAL] TODO: modify 'BaseDataset' for the Domain Generalization setting.
# Hint: combine the examples from the 3 source domains into a single 'examples' list
#class DomainGeneralizationDataset(Dataset):
#    def __init__(self, examples, transform):
#        self.examples = examples
#        self.T = transform
#
#    def __len__(self):
#        return len(self.examples)
#
#    def __getitem__(self, index):
#        x1, x2, x3 = self.examples[index]
#        x1, x2, x3 = self.T(x1), self.T(x2), self.T(x3)
#        targ_x = self.T(targ_x)
#        return x1, x2, x3

######################################################

class SeededDataLoader(DataLoader):
    def __init__(self, dataset: Dataset, batch_size=1, shuffle=None,
                 sampler=None,
                 batch_sampler=None,
                 num_workers=0, collate_fn=None,
                 pin_memory=False, drop_last=False, timeout=0,
                 worker_init_fn=None, multiprocessing_context=None,
                 generator=None, *, prefetch_factor=None, persistent_workers=False,
                 pin_memory_device=""):

        if not CONFIG.use_nondeterministic:
            def seed_worker(worker_id):
                worker_seed = torch.initial_seed() % 2**32
                np.random.seed(worker_seed)
                random.seed(worker_seed)

            generator = torch.Generator()
            generator.manual_seed(CONFIG.seed)

            worker_init_fn = seed_worker

        super().__init__(dataset, batch_size, shuffle, sampler, batch_sampler, num_workers, collate_fn,
                         pin_memory, drop_last, timeout, worker_init_fn, multiprocessing_context, generator,
                         prefetch_factor=prefetch_factor, persistent_workers=persistent_workers,
                         pin_memory_device=pin_memory_device)

#### PACS

In [7]:
def get_transform(size, mean, std, preprocess):
    transform = []
    if preprocess:
        transform.append(T.Resize(256))
        transform.append(T.RandomResizedCrop(size=size, scale=(0.7, 1.0)))
        transform.append(T.RandomHorizontalFlip())
    else:
        transform.append(T.Resize(size))
    transform.append(T.ToTensor())
    transform.append(T.Normalize(mean, std))
    return T.Compose(transform)


def load_data():
    CONFIG.num_classes = 7
    CONFIG.data_input_size = (3, 224, 224)

    # Create transforms
    mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225) # ImageNet Pretrain statistics
    train_transform = get_transform(size=224, mean=mean, std=std, preprocess=True)
    test_transform = get_transform(size=224, mean=mean, std=std, preprocess=False)

    # Load examples
    source_examples, target_examples = [], []
    if CONFIG.experiment in ['baseline', 'random', 'domain_adaptation']:
        # Load source
        f = open(os.path.join(CONFIG.dataset_args['text_root'], f"{CONFIG.dataset_args['source_domain']}.txt"), 'r')
        for line in f:
            line = line.strip().split()
            path, label = line[0].split('/')[0:], int(line[1])
            source_examples.append((os.path.join(CONFIG.dataset_args['images_root'], *path), label))
        f.close()

        # Load target
        f = open(os.path.join(CONFIG.dataset_args['text_root'], f"{CONFIG.dataset_args['target_domain']}.txt"), 'r')
        for line in f:
            line = line.strip().split()
            path, label = line[0].split('/')[0:], int(line[1])
            target_examples.append((os.path.join(CONFIG.dataset_args['images_root'], *path), label))
        f.close()

    # create Dataset object for the baseline and the Random Maps Ablation experiments
    if CONFIG.experiment in ['baseline', 'random']:
        train_dataset = BaseDataset(source_examples, transform=train_transform)
        test_dataset = BaseDataset(target_examples, transform=test_transform)

    # create Dataset object for the Domain Adaptation experiments
    if CONFIG.experiment in ['domain_adaptation']:
        train_dataset = DomainAdaptationDataset(source_examples, target_examples, transform=train_transform)
        test_dataset = BaseDataset(target_examples, transform=test_transform)

    ######################################################

    # Dataloaders
    train_loader = SeededDataLoader(
        train_dataset,
        batch_size=CONFIG.batch_size,
        shuffle=True,
        num_workers=CONFIG.num_workers,
        pin_memory=True,
        persistent_workers=True
    )

    test_loader = SeededDataLoader(
        test_dataset,
        batch_size=CONFIG.batch_size,
        shuffle=False,
        num_workers=CONFIG.num_workers,
        pin_memory=True,
        persistent_workers=True
    )

    return {'train': train_loader, 'test': test_loader}

### Models

#### Base ResNet 18

In [8]:
class BaseResNet18(nn.Module):
    def __init__(self):
        super(BaseResNet18, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)


    def forward(self, x):
        return self.resnet(x)

#### ASH ResNet 18

In [9]:
def get_activation_shaping_hook (mask):

    # Activation Shaping Module as a function that shall be hooked via 'register_forward_hook'
    # The hook captures mask variable from the parent scope, to update it,
    # remove() the hook and register a new one with the updated mask.

    def activation_shaping_hook(module, input, output):

        # print('ASH module registered on: ', module, 'mask shape: ', mask.shape, 'mask sum(): ', mask.sum())

        # binarize both activation map and mask using zero as threshold
        A_binary = torch.where(output<=0, torch.tensor(0.0), torch.tensor(1.0))
        M_binary = torch.where(mask<=0, torch.tensor(0.0), torch.tensor(1.0))

        # return the element-wise product of activation map and mask
        shaped_output = A_binary * M_binary
        return shaped_output

    return activation_shaping_hook

######################################################

# 'BaseResNet18' including the Activation Shaping Module
class ASHResNet18(nn.Module):
    def __init__(self):
        super(ASHResNet18, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)

        self.hook_handles = {}


    def register_activation_shaping_hook(self, layer_name, mask):
        hook = get_activation_shaping_hook(mask)

        for name, module in self.resnet.named_modules():
            if ((isinstance(module, nn.ReLU) or isinstance(module, nn.Conv2d) or isinstance(module, nn.BatchNorm2d)) and name == layer_name):
                if not name in self.hook_handles:
                    print('Insert activation shaping layer after ', name, module)
                    self.hook_handles[name] = module.register_forward_hook(hook)


    def remove_activation_shaping_hook(self, name):
        if name in self.hook_handles and self.hook_handles[name] is not None:
            handle = self.hook_handles[name]
            handle.remove()


    def forward(self, x):
        return self.resnet(x)

#### ASH ResNet 18 for Domain Adaptation

In [10]:
# 'BaseResNet18' including the Activation Shaping Module for Domain Adaptation
class ASHResNet18DomainAdaptation(nn.Module):
    def __init__(self, module_placement=None):
        super(ASHResNet18DomainAdaptation, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)

        self.module_placement = module_placement
        if module_placement==[]: self.module_placement = None
        self.activation_maps = {}
        self.activation_map_hook_handles = {}


    def get_extract_activation_map_hook1 (self, layer_name):
        def extract_activation_map_hook(module, input, output):
            self.activation_maps[layer_name] = output.clone().detach()
        return extract_activation_map_hook


    def get_extract_activation_map_hook2 (self, layer_name):
        def extract_activation_map_hook(module, input, output):
            A = torch.where(output<=0, torch.tensor(0.0), torch.tensor(1.0))
            M = self.activation_maps[layer_name]
            A = torch.where(M<=0, torch.tensor(0.0), torch.tensor(1.0))
            shaped_output = A * M
            return shaped_output
        return extract_activation_map_hook


    def forward(self, x, targ_x=None):
        if targ_x is not None: # First step for point 3
            if self.module_placement is not None:
                # Register a hook and perform a forward pass to store the target domain activation maps
                for layer_name, module in self.resnet.named_modules():
                    if (isinstance(module, nn.Conv2d) and layer_name in self.module_placement):
                        #print('Register a hook to store activation map of layer ', layer_name, module)
                        hook = self.get_extract_activation_map_hook1(layer_name)
                        self.activation_map_hook_handles[layer_name] = module.register_forward_hook(hook)
                _ = self.resnet(targ_x)
        else: # Second step for point 3
            if self.module_placement is not None:
                # Register a hook and perform a forward pass to compute source labels
                for layer_name, module in self.resnet.named_modules():
                    if (isinstance(module, nn.Conv2d) and layer_name in self.module_placement):
                        #print('Register a hook to store activation map of layer ', layer_name, module)
                        hook = self.get_extract_activation_map_hook2(layer_name)
                        self.activation_map_hook_handles[layer_name] = module.register_forward_hook(hook)
        output = self.resnet(x)

        return output

### Run

#### Evaluate

In [11]:
@torch.no_grad()
def evaluate(model, data):
    model.eval()

    acc_meter = Accuracy(task='multiclass', num_classes=CONFIG.num_classes)
    acc_meter = acc_meter.to(CONFIG.device)

    loss = [0.0, 0]
    for x, y in tqdm(data):
        with torch.autocast(device_type=CONFIG.device, dtype=torch.float16, enabled=True):
            x, y = x.to(CONFIG.device), y.to(CONFIG.device)
            logits = model(x)
            acc_meter.update(logits, y)
            loss[0] += F.cross_entropy(logits, y).item()
            loss[1] += x.size(0)

    accuracy = acc_meter.compute()
    loss = loss[0] / loss[1]
    logging.info(f'Accuracy: {100 * accuracy:.2f} - Loss: {loss}')
    print(f'Accuracy: {100 * accuracy:.2f} - Loss: {loss} \n')

#### Train

In [12]:
def train(model, data):
    # Create optimizers & schedulers
    optimizer = torch.optim.SGD(model.parameters(), weight_decay=0.0005, momentum=0.9, nesterov=True, lr=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=int(CONFIG.epochs * 0.8), gamma=0.1)
    scaler = torch.cuda.amp.GradScaler(enabled=True)

    # Load checkpoint (if it exists)
    cur_epoch = 0
    if os.path.exists(os.path.join('record', CONFIG.experiment_name, 'last.pth')):
        checkpoint = torch.load(os.path.join('record', CONFIG.experiment_name, 'last.pth'))
        cur_epoch = checkpoint['epoch']
        optimizer.load_state_dict(checkpoint['optimizer'])
        scheduler.load_state_dict(checkpoint['scheduler'])
        model.load_state_dict(checkpoint['model'])

    # Optimization loop
    for epoch in range(cur_epoch, CONFIG.epochs):
        model.train()

        for batch_idx, batch in enumerate(tqdm(data['train'])):

            # Compute loss
            with torch.autocast(device_type=CONFIG.device, dtype=torch.float16, enabled=True):

                if CONFIG.experiment in ['baseline']:
                    x, y = batch
                    x, y = x.to(CONFIG.device), y.to(CONFIG.device)
                    loss = F.cross_entropy(model(x), y)

                ######################################################

                elif CONFIG.experiment in ['random']:
                    x, y = batch
                    x, y = x.to(CONFIG.device), y.to(CONFIG.device)
                    loss = F.cross_entropy(model(x), y)

                elif CONFIG.experiment in ['domain_adaptation']:
                    src_x, src_y, targ_x = batch
                    x, y = src_x.to(CONFIG.device), src_y.to(CONFIG.device)
                    targ_x = targ_x.to(CONFIG.device)
                    loss = F.cross_entropy(model(x, targ_x=targ_x), y)

                ######################################################

            # Optimization step
            scaler.scale(loss / CONFIG.grad_accum_steps).backward()

            if ((batch_idx + 1) % CONFIG.grad_accum_steps == 0) or (batch_idx + 1 == len(data['train'])):
                scaler.step(optimizer)
                optimizer.zero_grad(set_to_none=True)
                scaler.update()

        scheduler.step()

        # Test current epoch
        logging.info(f'[TEST @ Epoch={epoch}]')
        evaluate(model, data['test'])

        # Save checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict(),
            'model': model.state_dict()
        }
        torch.save(checkpoint, os.path.join('record', CONFIG.experiment_name, 'last.pth'))

#### Main

In [13]:
layer_output_shapes = {
    'layer1.0.conv1': (64, 56, 56),         'layer1.0.relu': (64, 56, 56),          'layer1.0.conv2': (64, 56, 56),
    'layer1.1.conv1': (64, 56, 56),         'layer1.1.bn1': (64, 56, 56),           'layer1.1.relu': (64, 56, 56),          'layer1.1.conv2': (64, 56, 56),         'layer1.1.bn2': (64, 56, 56),
    'layer2.0.conv1': (128, 28, 28),        'layer2.0.relu': (128, 28, 28),         'layer2.0.conv2': (128, 28, 28),
    'layer2.1.conv1': (128, 28, 28),        'layer2.1.relu': (128, 28, 28),         'layer2.1.conv2': (128, 28, 28),
    'layer3.0.conv1': (256, 14, 14),        'layer3.0.relu': (256, 14, 14),         'layer3.0.conv2': (256, 14, 14),
    'layer3.1.conv1': (256, 14, 14),        'layer3.1.relu': (256, 14, 14),         'layer3.1.conv2': (256, 14, 14),
    'layer4.0.conv1': (512, 7, 7),          'layer4.0.relu': (512, 7, 7),           'layer4.0.conv2': (512, 7, 7),
    'layer4.1.conv1': (512, 7, 7),          'layer4.1.relu': (512, 7, 7),           'layer4.1.conv2': (512, 7, 7)
}


def create_random_mask(layer_name = 'layer4.1.conv2', mask_out_ratio = 0.0):
    # create a mask tensor with a given ratio of zeros
    print('mask_out_ratio: ',mask_out_ratio)
    layer_output_shape = layer_output_shapes[layer_name]

    rand_mat = torch.rand(layer_output_shape).to(CONFIG.device)
    mask = torch.where(rand_mat <= mask_out_ratio, 0.0, 1.0).to(CONFIG.device)
    return mask


def main ():
    # Load dataset
    data = load_data()

    # Load model
    if CONFIG.experiment in ['baseline']:
        model = BaseResNet18()

    ######################################################

    elif CONFIG.experiment in ['random']:
        model = ASHResNet18()
        module_placement = CONFIG.experiment_args['module_placement']
        mask_out_ratio = CONFIG.experiment_args['mask_out_ratio']

        for layer_name in module_placement:
            random_mask = create_random_mask(layer_name = layer_name, mask_out_ratio = mask_out_ratio)
            print('layer_name', layer_name, 'random_mask sum(): ', random_mask.sum())
            model.register_activation_shaping_hook(layer_name, random_mask)

    elif CONFIG.experiment in ['domain_adaptation']:
        module_placement = CONFIG.experiment_args['module_placement']
        model = ASHResNet18DomainAdaptation(module_placement)

    ######################################################

    model.to(CONFIG.device)

    if not CONFIG.test_only:
        train(model, data)
    else:
        evaluate(model, data['test'])


## 0 - Reproduce the Baseline

### 0.1 - Baseline

#### 0.1.1 - Art painting &rightarrow; Cartoon

In [14]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'baseline',
    'experiment_name': '0_1_1',
    'experiment_args': '{}',
    'dataset_args': { 'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='baseline', experiment_name='0_1_1', experiment_args='{}', dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 92.5MB/s]
100%|██████████| 19/19 [00:09<00:00,  1.97it/s]


Accuracy: 30.89 - Loss: 0.014743562139341856 



100%|██████████| 19/19 [00:06<00:00,  2.80it/s]


Accuracy: 41.98 - Loss: 0.013025202340233449 



...

Accuracy: 54.39 - Loss: 0.010741157272875106 



100%|██████████| 19/19 [00:06<00:00,  2.90it/s]


Accuracy: 54.39 - Loss: 0.010649757284938069 



100%|██████████| 19/19 [00:05<00:00,  3.31it/s]


Accuracy: 54.61 - Loss: 0.01065421475688752 



#### 0.1.2 - Art painting &rightarrow; Sketch

In [15]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'baseline',
    'experiment_name': '0_1_2',
    'experiment_args': '{}',
    'dataset_args': { 'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='baseline', experiment_name='0_1_2', experiment_args='{}', dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)


100%|██████████| 31/31 [00:12<00:00,  2.51it/s]


Accuracy: 10.56 - Loss: 0.016417324103056612 



100%|██████████| 31/31 [00:10<00:00,  2.85it/s]


Accuracy: 24.08 - Loss: 0.01549151532975855 



...

Accuracy: 39.88 - Loss: 0.015360646528213435 



100%|██████████| 31/31 [00:10<00:00,  3.05it/s]


Accuracy: 40.09 - Loss: 0.015157393278678764 



100%|██████████| 31/31 [00:10<00:00,  2.87it/s]


Accuracy: 40.37 - Loss: 0.015194548453711832 



#### 0.1.3 - Art painting &rightarrow; Photo

In [16]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'baseline',
    'experiment_name': '0_1_3',
    'experiment_args': '{}',
    'dataset_args': { 'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='baseline', experiment_name='0_1_3', experiment_args='{}', dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)


100%|██████████| 14/14 [00:05<00:00,  2.78it/s]


Accuracy: 60.84 - Loss: 0.011620395126456986 



100%|██████████| 14/14 [00:04<00:00,  3.30it/s]


Accuracy: 79.64 - Loss: 0.0066013281930706455 



...

Accuracy: 95.87 - Loss: 0.0010738102981296485 



100%|██████████| 14/14 [00:04<00:00,  3.22it/s]


Accuracy: 95.81 - Loss: 0.001084593999127071 



100%|██████████| 14/14 [00:04<00:00,  3.31it/s]


Accuracy: 95.93 - Loss: 0.0010711053720937518 



## 1 - Activation Shaping module

### 1.1 - ASH module after last convolutional layer

#### 1.1.1 - Art Painting &rightarrow; Cartoon

In [17]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_1_1',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='1_1_1', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
layer_name layer4.1.conv2 random_mask sum():  tensor(22611., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 19/19 [00:06<00:00,  2.73it/s]


Accuracy: 19.97 - Loss: 0.015888305094868656 



100%|██████████| 19/19 [00:06<00:00,  2.77it/s]


Accuracy: 27.82 - Loss: 0.014835419355805823 



...

Accuracy: 43.60 - Loss: 0.011582614720477993 



100%|██████████| 19/19 [00:06<00:00,  3.11it/s]


Accuracy: 43.30 - Loss: 0.011567086133940635 



100%|██████████| 19/19 [00:05<00:00,  3.38it/s]


Accuracy: 43.26 - Loss: 0.011544538922480755 



#### 1.1.2 - Art Painting &rightarrow; Sketch

In [18]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_1_2',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='1_1_2', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
layer_name layer4.1.conv2 random_mask sum():  tensor(22611., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 31/31 [00:11<00:00,  2.71it/s]


Accuracy: 6.39 - Loss: 0.017047369683416906 



100%|██████████| 31/31 [00:10<00:00,  2.93it/s]


Accuracy: 10.23 - Loss: 0.016052489579254873 



...

Accuracy: 38.23 - Loss: 0.014108155331377402 



100%|██████████| 31/31 [00:10<00:00,  3.01it/s]


Accuracy: 38.20 - Loss: 0.014115967802735618 



100%|██████████| 31/31 [00:10<00:00,  2.88it/s]


Accuracy: 38.08 - Loss: 0.014157110390453188 



#### 1.1.3 - Art Painting &rightarrow; Photo

In [19]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_1_3',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='1_1_3', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
layer_name layer4.1.conv2 random_mask sum():  tensor(22611., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 14/14 [00:04<00:00,  2.89it/s]


Accuracy: 33.35 - Loss: 0.014961894043905292 



100%|██████████| 14/14 [00:04<00:00,  3.30it/s]


Accuracy: 47.66 - Loss: 0.012348066903873831 



...

Accuracy: 91.38 - Loss: 0.0030863082783664773 



100%|██████████| 14/14 [00:04<00:00,  3.35it/s]


Accuracy: 91.32 - Loss: 0.003098668809422476 



100%|██████████| 14/14 [00:04<00:00,  3.36it/s]


Accuracy: 91.56 - Loss: 0.0030862670681790676 



### 1.2 - ASH module after all convolutional layers

#### 1.2.1 - Art Painting &rightarrow; Cartoon

In [20]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_2_1',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='1_2_1', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
layer_name layer1.0.conv1 random_mask sum():  tensor(180587., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.1
layer_name layer1.0.conv2 random_mask sum():  tensor(180685., device='cuda:0')
Insert activation shaping layer 

100%|██████████| 19/19 [00:06<00:00,  2.86it/s]


Accuracy: 15.10 - Loss: 0.015846378719847357 



100%|██████████| 19/19 [00:06<00:00,  2.72it/s]


Accuracy: 16.25 - Loss: 0.01595945066356008 



...

Accuracy: 21.67 - Loss: 0.015260613717316768 



100%|██████████| 19/19 [00:07<00:00,  2.69it/s]


Accuracy: 22.18 - Loss: 0.015294040836164976 



100%|██████████| 19/19 [00:06<00:00,  2.77it/s]


Accuracy: 20.31 - Loss: 0.0153128067381146 



#### 1.2.2 - Art Painting &rightarrow; Sketch

In [21]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_2_2',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='1_2_2', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
layer_name layer1.0.conv1 random_mask sum():  tensor(180587., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.1
layer_name layer1.0.conv2 random_mask sum():  tensor(180685., device='cuda:0')
Insert activation shaping layer a

100%|██████████| 31/31 [00:11<00:00,  2.60it/s]


Accuracy: 12.09 - Loss: 0.016710730811420303 



100%|██████████| 31/31 [00:11<00:00,  2.69it/s]


Accuracy: 15.81 - Loss: 0.01636714569998625 



...

Accuracy: 16.75 - Loss: 0.015605378405803274 



100%|██████████| 31/31 [00:10<00:00,  2.86it/s]


Accuracy: 19.09 - Loss: 0.015553225873838887 



100%|██████████| 31/31 [00:11<00:00,  2.73it/s]


Accuracy: 16.44 - Loss: 0.015723180637374184 



#### 1.2.3 - Art Painting &rightarrow; Photo

In [22]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_2_3',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='1_2_3', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
layer_name layer1.0.conv1 random_mask sum():  tensor(180587., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.1
layer_name layer1.0.conv2 random_mask sum():  tensor(180685., device='cuda:0')
Insert activation shaping layer af

100%|██████████| 14/14 [00:05<00:00,  2.78it/s]


Accuracy: 14.19 - Loss: 0.01644154472979243 



100%|██████████| 14/14 [00:04<00:00,  3.08it/s]


Accuracy: 15.03 - Loss: 0.016415861338198542 



...

Accuracy: 25.57 - Loss: 0.015519935142494248 



100%|██████████| 14/14 [00:04<00:00,  3.14it/s]


Accuracy: 27.90 - Loss: 0.015364291710767917 



100%|██████████| 14/14 [00:04<00:00,  3.07it/s]


Accuracy: 28.14 - Loss: 0.015488921168321622 



### 1.3 - ASH module after every three convolutional layers

#### 1.3.1 - Art Painting &rightarrow; Cartoon

In [23]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_3_1',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='1_3_1', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
layer_name layer1.0.conv1 random_mask sum():  tensor(180587., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.1
layer_name layer1.1.conv2 random_mask sum():  tensor(180685., device='cuda:0')
Insert activation shaping layer after  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.1
layer_name layer2.1.conv1 random_mask sum():  tensor(90235.

100%|██████████| 19/19 [00:06<00:00,  2.86it/s]


Accuracy: 11.95 - Loss: 0.016639337401341252 



100%|██████████| 19/19 [00:06<00:00,  2.93it/s]


Accuracy: 21.72 - Loss: 0.015651013002867586 



...

Accuracy: 30.63 - Loss: 0.015301098900850315 



100%|██████████| 19/19 [00:05<00:00,  3.28it/s]


Accuracy: 31.10 - Loss: 0.015274142852822262 



100%|██████████| 19/19 [00:05<00:00,  3.28it/s]


Accuracy: 31.48 - Loss: 0.015209068312173004 



#### 1.3.2 - Art Painting &rightarrow; Sketch

In [24]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_3_2',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='1_3_2', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
layer_name layer1.0.conv1 random_mask sum():  tensor(180587., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.1
layer_name layer1.1.conv2 random_mask sum():  tensor(180685., device='cuda:0')
Insert activation shaping layer after  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.1
layer_name layer2.1.conv1 random_mask sum():  tensor(90235.,

100%|██████████| 31/31 [00:11<00:00,  2.64it/s]


Accuracy: 16.80 - Loss: 0.01707930205644436 



100%|██████████| 31/31 [00:10<00:00,  2.90it/s]


Accuracy: 17.49 - Loss: 0.015850286503182806 



...

Accuracy: 24.61 - Loss: 0.014571800945550923 



100%|██████████| 31/31 [00:11<00:00,  2.81it/s]


Accuracy: 23.19 - Loss: 0.01471752136164447 



100%|██████████| 31/31 [00:10<00:00,  2.85it/s]


Accuracy: 23.11 - Loss: 0.014602089328722178 



#### 1.3.3 - Art Painting &rightarrow; Photo

In [25]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_3_3',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='1_3_3', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
layer_name layer1.0.conv1 random_mask sum():  tensor(180587., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.1
layer_name layer1.1.conv2 random_mask sum():  tensor(180685., device='cuda:0')
Insert activation shaping layer after  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.1
layer_name layer2.1.conv1 random_mask sum():  tensor(90235., 

100%|██████████| 14/14 [00:05<00:00,  2.63it/s]


Accuracy: 14.07 - Loss: 0.017098118016819753 



100%|██████████| 14/14 [00:04<00:00,  2.81it/s]


Accuracy: 34.31 - Loss: 0.014474241390913547 



...

Accuracy: 65.93 - Loss: 0.0080231504525967 



100%|██████████| 14/14 [00:05<00:00,  2.65it/s]


Accuracy: 66.41 - Loss: 0.008023632643465511 



100%|██████████| 14/14 [00:05<00:00,  2.62it/s]


Accuracy: 66.41 - Loss: 0.007960782490090695 



## 2 - Random Activation Maps

### 2.1 - ASH module after last convolutional layer

#### 2.1.1 - Art Painting &rightarrow; Cartoon, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_1',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.1.2 - Art Painting &rightarrow; Sketch, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_2',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.1.3 - Art Painting &rightarrow; Photo, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_3',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.1.4 - Art Painting &rightarrow; Cartoon, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_4',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.1.5 - Art Painting &rightarrow; Sketch, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_5',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.1.6 - Art Painting &rightarrow; Photo, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_6',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.1.7 - Art Painting &rightarrow; Cartoon, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_7',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.1.8 - Art Painting &rightarrow; Sketch, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_8',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.1.9 - Art Painting &rightarrow; Photo, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_9',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

### 2.2 - ASH module after every convolutional layer

#### 2.2.1 - Art Painting &rightarrow; Cartoon, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_1',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.2.2 - Art Painting &rightarrow; Sketch, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_2',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.2.3 - Art Painting &rightarrow; Photo, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_3',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.2.4 - Art Painting &rightarrow; Cartoon, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_4',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.2.5 - Art Painting &rightarrow; Sketch, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_5',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.2.6 - Art Painting &rightarrow; Photo, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_6',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.2.7 - Art Painting &rightarrow; Cartoon, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_7',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.2.8 - Art Painting &rightarrow; Sketch, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_8',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.2.9 - Art Painting &rightarrow; Photo, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_9',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

### 2.3 - ASH module after every three convolutional layers

#### 2.3.1 - Art Painting &rightarrow; Cartoon, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_1',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.3.2 - Art Painting &rightarrow; Sketch, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_2',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.3.3 - Art Painting &rightarrow; Photo, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_3',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.3.4 - Art Painting &rightarrow; Cartoon, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_4',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.3.5 - Art Painting &rightarrow; Sketch, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_5',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.3.6 - Art Painting &rightarrow; Photo, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_6',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.3.7 - Art Painting &rightarrow; Cartoon, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_7',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.3.8 - Art Painting &rightarrow; Sketch, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_8',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.3.9 - Art Painting &rightarrow; Photo, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_9',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

## 3 - Adapting Activation Maps across Domains

### 3.1 - module placement: layer2.1.conv2

#### 3.1.1 - Art Painting &rightarrow; Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'domain_adaptation',
    'experiment_name': '3_1_1',
    'experiment_args': {'module_placement': ['layer2.1.conv2']},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 3.1.2 - Art Painting &rightarrow; Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'domain_adaptation',
    'experiment_name': '3_1_2',
    'experiment_args': {'module_placement': ['layer2.1.conv2']},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 3.1.3 - Art Painting &rightarrow; Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'domain_adaptation',
    'experiment_name': '3_1_3',
    'experiment_args': {'module_placement': ['layer2.1.conv2']},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

## Ext. 2 - Binarization Ablation